## Assignment #4
David Perry - sez326

Chapter 5 Resampling Methods – Applied 

Exercises 3, 5, 6, & 9

## 3.  We now review k-fold cross-validation.

(a)	Explain how k-fold cross validation is implemented.

The k-fold cross-validation algorithm is a method to estimate the test error of a given statistical estimator.  One starts by taking a training set (of size N) and partitioning (or dividing it) into k non-overlapping equal parts.  If k does not divide N evenly, it is split into k folds as “equal” as possible.  For the sake of making sure it is understood, let’s say N = 1003, and k = 10, there will be 3 folds with 101 elements and the remaining 7 folds will have 100 elements each.  The model is then fitted k times, with each time leaving out a different fold as a validation set.  For each of the k model fits, the left our fold is used to calculate the validation error.  Lastly, the average of the k validation errors is the estimate of the test error.

(b)	What are the advantages and disadvantages of k-fold cross validation relative to:

i.	The validation set approach?
Compared to k-fold cross validation, the validation set approach requires less effort computationally since it only fits the model once.  Overall, it is simpler and easier to implement.  A drawback, however, is that the validation set approach tends to overestimate the test error since it only uses half of the sample to fit the model.  Generally speaking, a larger sample size leads to a lower test error.  Additionally, fitting only half the model will make the test error estimate dependent on the 50% of the sample we choose.  Both of these aspects are also true for k-fold cross-validation, but the effect is much smaller.  This is shown in Chapter 5 in figures 5.2 and 5.4 of the test. 

ii.	LOOCV
Leave-one-out cross-validation (LOOCV) has less bias than k-fold cross validation since it uses almost all of the points of the data set, nearly unbiased.  Conversely, this is yet another instance of the bias-variance trade-off, and LOOCV will have more variance than k-fold cross validation.  This happens when when every pair of the n-1 fitted models differs only by 2 points, making the validation errors extremely correlated given they share n-2 of the n-1 points used for each fit.  With k-fold cross validation, this correlation is greatly diminished if k=5 or k=10, for instance.

Consider a k=10 example: each pair of the 10 model “samples” share only about 80% of the points.  With k=5, each pair of the 5 models “samples” share approximately 60% of the points.  Additionally, k-fold cross validation is less computationally intensive, requiring only k model fits instead of the n fits with LOOCV.  As pointed out in Equation 5.2, there’s an exception to this case for the least squares linear or polynomial regression – in this case LOOCV requires only the same computation as a single model fit.


In [218]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from IPython.display import display, HTML
import statsmodels.api as sm
import statsmodels.formula.api as smf
#from sklearn import datasets
from scipy import stats

# Load data
from ISLP import load_data

In [195]:
def confusion_table(confusion_mtx):
    """Renders a nice confusion table with labels"""
    confusion_df = pd.DataFrame({'y_pred=0': np.append(confusion_mtx[:, 0], confusion_mtx.sum(axis=0)[0]),
                                 'y_pred=1': np.append(confusion_mtx[:, 1], confusion_mtx.sum(axis=0)[1]),
                                 'Total': np.append(confusion_mtx.sum(axis=1), ''),
                                 '': ['y=0', 'y=1', 'Total']}).set_index('')
    return confusion_df

def total_error_rate(confusion_matrix):
    """Derive total error rate from confusion matrix"""
    return 1 - np.trace(confusion_mtx) / np.sum(confusion_mtx)

### 5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

In [196]:
default_df = load_data("Default")
default_df = default_df.reset_index().drop('index', axis=1)

# Check for missing
assert default_df.isna().sum().sum() == 0

# Rationalise types
default_df = pd.get_dummies(default_df, dtype=np.float64).drop(['default_No', 'student_No'], axis=1)

display(default_df.head())

,balance,income,default_Yes,student_Yes
0,729.526495,44361.625074,0.0,0.0
1,817.180407,12106.134700,0.0,1.0
2,1073.549164,31767.138947,0.0,0.0
3,529.250605,35704.493935,0.0,0.0
4,785.655883,38463.495879,0.0,0.0


### (a) Fit a logistic regression model that uses income and balance to predict default.
### (b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

- i. Split the sample set into a training set and a validation set.
- ii. Fit a multiple logistic regression model using only the training observations.
- iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.
- iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

### (c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [197]:
for s in range(1,4):
    display(HTML('<h3>Random seed = {}</h3>'.format(s)))
    # Create index for 50% holdout set
    np.random.seed(s)
    train = np.random.rand(len(default_df)) < 0.5
    
    response   = 'default_Yes'
    predictors = ['income', 'balance']
    
    X_train = np.array(default_df[train][predictors])
    X_test  = np.array(default_df[~train][predictors])
    y_train = np.array(default_df[train][response])
    y_test  = np.array(default_df[~train][response])
    
    # Logistic regression
    logit       = LogisticRegression()
    model_logit = logit.fit(X_train, y_train)
    
    # Predict
    y_pred = model_logit.predict(X_test)
    
    # Analysis
    confusion_mtx = confusion_matrix(y_test, y_pred)
    display(confusion_table(confusion_mtx))
    
    total_error_rate_pct = np.around(total_error_rate(confusion_mtx) * 100, 4)
    print('The_total_error_rate_is: {}%'.format(total_error_rate_pct))


,y_pred=0,y_pred=1,Total
,,,
y=0,4846,0,4846
y=1,164,0,164
Total,5010,0,


The_total_error_rate_is: 3.2735%


,y_pred=0,y_pred=1,Total
,,,
y=0,4691,1,4692
y=1,176,0,176
Total,4867,1,


The_total_error_rate_is: 3.636%


,y_pred=0,y_pred=1,Total
,,,
y=0,4773,1,4774
y=1,163,0,163
Total,4936,1,


The_total_error_rate_is: 3.3219%


### (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [198]:
for s in range(1,4):
    display(HTML('<h3>Random seed = {}</h3>'.format(s)))
    # Create index for 50% holdout set
    np.random.seed(s)
    train = np.random.rand(len(default_df)) < 0.5
    
    response   = 'default_Yes'
    predictors = ['income', 'balance', 'student_Yes']
    
    X_train = np.array(default_df[train][predictors])
    X_test  = np.array(default_df[~train][predictors])
    y_train = np.array(default_df[train][response])
    y_test  = np.array(default_df[~train][response])
    
    # Logistic regression
    logit       = LogisticRegression()
    model_logit = logit.fit(X_train, y_train)
    
    # Predict
    y_pred = model_logit.predict(X_test)
    
    # Analysis
    confusion_mtx = confusion_matrix(y_test, y_pred)
    display(confusion_table(confusion_mtx))
    
    total_error_rate_pct = np.around(total_error_rate(confusion_mtx) * 100, 4)
    print('The_total_error_rate_is: {}%'.format(total_error_rate_pct))

,y_pred=0,y_pred=1,Total
,,,
y=0,4846,0,4846
y=1,164,0,164
Total,5010,0,


The_total_error_rate_is: 3.2735%


,y_pred=0,y_pred=1,Total
,,,
y=0,4691,1,4692
y=1,176,0,176
Total,4867,1,


The_total_error_rate_is: 3.636%


,y_pred=0,y_pred=1,Total
,,,
y=0,4773,1,4774
y=1,163,0,163
Total,4936,1,


The_total_error_rate_is: 3.3219%


**Comment**

It is difficult to discern if the student predictor has improved the model because of the variation in results.

### 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the glm() function. Do not forget to set a random seed before beginning your analysis.

### (a) Using the summary() and glm() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [199]:
response   = 'default_Yes'
predictors = ['income', 'balance']

X_all = sm.add_constant(np.array(default_df[predictors]))
y_all = np.array(default_df[response])

## Logistic regression
model_logit = sm.Logit(y_all, X_all).fit(disp=False);    

# Summary
print(model_logit.summary())

statsmodels_est = pd.DataFrame({'coef_sm': model_logit.params, 'SE_sm': model_logit.bse})
display(statsmodels_est)

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Wed, 20 Mar 2024   Pseudo R-squ.:                  0.4594
Time:                        20:48:07   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
x1          2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
x2             0.0056      0.000     24.835      0.0

,coef_sm,SE_sm
0,-11.540468,0.434772
1,0.000021,0.000005
2,0.005647,0.000227


### (b) Write a function, boot.fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [200]:
def boot_fn(df, idx):
    response   = 'default_Yes'
    predictors = ['income', 'balance']
    
    X = sm.add_constant(np.array(df[predictors].loc[idx]));
    y = np.array(df[response].loc[idx]) 
       
    # Logistic regression
    model_logit = sm.Logit(y, X).fit(disp=False);  
    return model_logit.params;

### (c) Use the boot() function together with your boot.fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.

In [201]:
def boot_idx(n):
    """Return index for bootstrap sample of size n
    e.g. generate array in range 0 to n, with replacement"""
    return np.random.randint(low=0, high=n, size=n)

def boot(fn, data_df, samples):
    """Perform bootstrap for B number of samples"""
    results = []
    for s in range(samples):
        Z = fn(data_df, boot_idx(data_df.shape[0]))
        results += [Z]
    return np.array(results)

def standard_deviation(X):
    """Compute deviation error for jth element in matrix X
    equivalent to np.std(X, axis=0)"""
    X_bar = np.mean(X, axis=0)
    SE = np.sqrt((np.sum(np.square(X - X_bar), axis=0)) / (len(X)))
    return SE

B = 10000
coef_preds    = boot(boot_fn, default_df, samples=B)
coef_pred     = np.mean(coef_preds, axis=0)
standard_errs = standard_deviation(coef_preds)

bootstrap_est = pd.DataFrame({'coef_boot': coef_pred, 'SE_boot': standard_errs})
display(bootstrap_est)

,coef_boot,SE_boot
0,-11.565762,0.434150
1,0.000021,0.000005
2,0.005660,0.000228


### (d) Comment on the estimated standard errors obtained using the glm() function and using your bootstrap function.

In [202]:
pd.concat([statsmodels_est, bootstrap_est], axis=1)

,coef_sm,SE_sm,coef_boot,SE_boot
0,-11.540468,0.434772,-11.565762,0.434150
1,0.000021,0.000005,0.000021,0.000005
2,0.005647,0.000227,0.005660,0.000228


Let's compare the standard errors estimated by the statsmodels (_sm) summary() function with estimates obtained by bootstrap (_boot) in the table above. The standard errors for x1 and x2 (rows 1 and 2) are indistinguishable to 6 decimal places. The coefficient for x2 and the statistics for the intercept x0 vary slightly.

Note that the disparity is slightly more significant when fewer bootstrap samples are used. Here 10,000 were used, but the estimates were alike to within the same order of magnitude with only 10 bootstrap samples.

#**QUESTION:** Why are the standard errors provided by statsmodels equivalent to the standard deviations by my calculations, when $SE = \frac{σ}{\sqrt{n}}$ ?

## 9. We will now consider the Boston housing data set, from the ISLP library.

In [203]:
# Load data
#import pandas as pd
#import numpy as np
from ISLP import load_data
boston = load_data("Boston")

In [204]:
type(boston)

pandas.core.frame.DataFrame

In [205]:
boston.keys()

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'lstat', 'medv'],
      dtype='object')

In [206]:
print(boston.shape)

(506, 13)


In [207]:
pd.DataFrame(boston).head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


### (a) Based on this data set, provide an estimate for the population mean of medv. Call this estimate μˆ.

In [208]:
print('The Estimated Population Mean is:', np.mean(boston['medv'])) 

The Estimated Population Mean is: 22.532806324110677


### (b) Provide an estimate of the standard error of μˆ. Interpret this result.

Hint: The standard error of the sample mean can be computed by dividing the sample standard deviation by the square root of the number of observations.

In [209]:
print('The Estimation of the Standard Error is:', np.std(boston['medv']) / np.sqrt(len(boston)))

The Estimation of the Standard Error is: 0.4084569346972867


### (c) Now estimate the standard error of μˆ using the bootstrap. How does this compare to your answer from (b)?

In [210]:
#c
#bootstrap standard deviation of mean
def boot(boston,R):
    medv = []
    num_samples = 500
    for i in range(R):
        indices = np.random.choice(boston.index, num_samples, replace=True)
        medv.append(boston['medv'].loc[indices].mean())
    bootstrap_statistics = {'estimated_error_value':np.mean(medv),'std_dev':np.std(medv)}   
    return bootstrap_statistics

In [211]:
result = boot(boston, 1000)
print('The Bootstrap Results for the Estimated Error and the Standard Deviation are:', result)

The Bootstrap Results for the Estimated Error and the Standard Deviation are: {'estimated_error_value': 22.551438, 'std_dev': 0.4004256010746566}


The bootstrap method gives a remarkably good estimate of the standard error, when compared to the same estimate derived analytically. The bootstrap approach is computationally more expensive, but has the advantage that no analytic derivation of the standard error for the statistic is required, making it much more general for application to other statistics.

### (d) Based on your bootstrap estimate from (c), provide a 95 % confidence interval for the mean of medv.

In [212]:
mean = result['estimated_error_value']
std = result['std_dev']

print('With 95% confidence, we can say that the interval is as follows: [{},{}]'.format(mean - 2*std,mean+ 2*std))

With 95% confidence, we can say that the interval is as follows: [21.750586797850687,23.352289202149315]


In [213]:
print('Estimated Value for the Median is: ',np.median(boston['medv']))

Estimated Value for the Median is:  21.2


### (e) Based on this dataset, provide an estimate, μˆmed, for the median value of medv in the population.
     &
### (f) We now would like to estimate the standard error of μˆmed. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.

In [214]:
#e & #f
def boot(boston,R):
    median = []
    num_samples = 500
    for i in range(R):
        indices = np.random.choice(boston.index, num_samples, replace=True)
        median.append(boston['medv'].loc[indices].median())
    bootstrap_statistics = {'estimated_value':np.mean(median),'std_dev':np.std(median)}   
    return bootstrap_statistics

In [215]:
result = boot(boston,1000)
print('Boostrap Results for the Median are:', result)

Boostrap Results for the Median are: {'estimated_value': 21.19475, 'std_dev': 0.385415279276782}


The estimated standard error for the median value of medv is similar to the estimated standard error for the mean, which is not suprising as we would expect the precision of median and and mean to be related.

### g) Based on this data set, provide an estimate for the tenth percentile of medv in Boston suburbs. Call this quantity μˆ0.1. (You can use the quantile() function.)

In [216]:
print('The Estimate for the 10th Quantile is:',boston['medv'].quantile(0.1))

The Estimate for the 10th Quantile is: 12.75


### (h) Use the bootstrap to estimate the standard error of μˆ0.1. Comment on your findings.

In [217]:
def boot(boston,R):
    tenth_quantiles = []
    num_samples = 500
    for i in range(R):
        indices = np.random.choice(boston.index, num_samples, replace=True)
        tenth_quantiles.append(boston['medv'].loc[indices].quantile(0.1))
    bootstrap_statistics = {'estimated_value':np.mean(tenth_quantiles),'std_dev':np.std(tenth_quantiles)}   
    return bootstrap_statistics

result = boot(boston,1000)
print('The Boostrap Results for the Median are:', result)

The Boostrap Results for the Median are: {'estimated_value': 12.73135, 'std_dev': 0.5255567310005647}
